# ⚖️ Vizualizace pro evaluaci modelů

V tomto notebooku si ukážeme základní vizualizace pro vyhodnocení úspěšnosti modelů strojového učení.✨

In [ ]:
# plotting
import matplotlib.pyplot as plt

# models
from sklearn.tree import DecisionTreeClassifier 

# metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay

# data generation and manipulation
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd

## Evaluace klasifikátoru
V přednášce jsme si zmínili, že klasifikátor můžeme vyhodnotit například pomocí matice záměn (ať už klasické nebo normalizované) nebo pomocí ROC křivky.

### 🔲 Matice záměn
K jejímu vykreslení můžeme použít funkci `ConfusionMatrixDisplay.from_predictions` z balíčku `sklearn`. Jako parametr potřebujeme zadat reálné a predikované hodnoty, ze kterých se matice vypočítá. Volitelně můžeme zadat například barevnou mapu (parametr `cmap`) nebo `Axes` objekt, do kterého chceme matici vykreslit. Pokud chceme, aby byla matice normalizovaná, je třeba nastavit parametr `normalize` na 'true'. Více si můžete přečíst v dokumentaci.

🗂 [dokumentace](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html#sklearn.metrics.ConfusionMatrixDisplay.from_predictions)

👨🏽‍💻 [user guide](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix)

In [ ]:
def show_confusion_matrices(y_real, y_pred):
    fig, axs = plt.subplots(1,2, figsize=(16,6))

    ConfusionMatrixDisplay.from_predictions(y_real, y_pred, ax=axs[0], cmap='RdPu')
    ConfusionMatrixDisplay.from_predictions(y_real, y_pred, ax=axs[1], cmap='RdPu', normalize='true')

    axs[0].set_title('Confusion matrix')
    axs[1].set_title('Normalized confusion matrix')

### Matice záměn pro binární klasifikaci 🟢 🟣 

In [ ]:
X, y = make_classification(n_classes=2, n_clusters_per_class=2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

show_confusion_matrices(y_test, y_pred)

### Matice záměn pro ternární klasifikaci 🟠 🟢 🟣

In [ ]:
X, y = make_classification(n_classes=3, n_clusters_per_class=1, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

show_confusion_matrices(y_test, y_pred)

### 📈 ROC křivka
ROC křivku vykreslíme pomocí funkce `RocCurveDisplay.from_estimator` balíčku `sklearn`. Stačí zadat klasifikátor a testovací data (matici příznaků a vektor vysvětlované proměnné). Volitelně můžeme zadat i `Axes` objekt, do kterého se má výsledný graf vykreslit.

In [ ]:
from sklearn.svm import SVC

X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# SVC is a classification model (https://scikit-learn.org/stable/modules/svm.html)
clf = SVC(random_state=0, probability=True).fit(X_train, y_train)
#clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

fig, ax = plt.subplots()
_ = RocCurveDisplay.from_estimator(clf, X_test, y_test, ax=ax)

Křivku ale můžeme vytvořit i sami. Z přednášky víme, že nás zajímají hodnoty TPR a FPR vzhledem k určitému tresholdu. Ty dokážeme získat pomocí funkce `roc_curve` z `sklearn.metrics`. Jako parametr jí je třeba zadat vektor vysvětlované proměnné a vektor pravděpodobností, že dané pozorování patří do pozitivní třídy (tedy že je hodnota vysvětlované proměnné 1).

Vektor vysvětlované proměnné už máme - `y_test`. Vektor pravděpodobností získáme pomocí funkce `predict_proba`.

🗂 [dokumentace](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba)

In [ ]:
clf.predict_proba(X_test)

Výstupem funkce je pole dvojic pravděpodobností pro každé pozorování z X_test. První element vyjadřuje pravděpodobnost, že pozorování patří do třídy 0 a druhý zase pravděpodobnost, že pozorování patří do třídy 1 (můžete si ověřit, že se obě čísla sčítají na 1).

Nás zajímá pravděpodobnost, že pozorování patří do třídy 1, tedy druhý element. Vektor dostaneme následovně.

In [ ]:
probabilities = clf.predict_proba(X_test)[:,1]

Teď už máme oba potřebné parametry do funkce `roc_curve`. Ta nám vrátí TPR a FPR. 

🗂 [dokumentace](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, probabilities)

# Increasing fpr/tpr such that element i is 
# the fpr/tpr of predictions with score >= thresholds[i].

In [ ]:
print('FPR')
print(['{:0.3f}'.format(x) for x in fpr])
print('TPR')
print(['{:0.3f}'.format(x) for x in tpr])

S FPR a TPR už umíme vykreslit ROC křivku 🥳.

In [ ]:
# styling
blue = '#8592dc'
violet = '#9047A0'
red = '#d14081'
grey = '#8E8DB4'

plt.rcParams.update({"axes.grid" : True})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=[blue, violet, red, grey])
plt.style.use('seaborn-darkgrid')

In [ ]:
# custom ROC curve visualization
fig, ax = plt.subplots(figsize=(16,8))

# ROC curve
ax.plot(fpr, tpr, label='ROC')
# area under the curve
ax.fill_between(fpr, tpr, alpha=0.2, label='AUC = {:.3f}'.format(roc_auc_score(y_test, probabilities)))
# random classifier
ax.plot([0,1], [0,1], '--', label='Random classifier')

ax.legend()
ax.set_xlabel('False positive rate (positive label: 1)')
ax.set_ylabel('True positive rate (positive label: 1)')

_ = ax.set_title('ROC curve')

Co jsme si doposud nezmínili - na grafu vidíme znázorněno i **AUC** (angl. Area Under Curve). Jedná se o souhrnné hodnocení klasifikační schopnosti proměnné, které je vyjádřeno plochou pod ROC křivkou. Čím je AUC blíže 1, tím jde o lepší model. Její hodnotu můžeme získat pomocí `metrics.roc_auc_score`.

ROC křivku můžete použít i pro vyhodnocení **klasifikace do více tříd**, vizte [dokumentaci](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#plot-roc-curves-for-the-multiclass-problem). Nebo tak můžete **porovnat více klasifikátorů** (☝️ kód si můžete zkusit přepsat s využitím `Axes`).

In [ ]:
clf2 = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

svc_disp = RocCurveDisplay.from_estimator(clf, X_test, y_test)
rfc_disp = RocCurveDisplay.from_estimator(clf2, X_test, y_test, ax=svc_disp.ax_)
plt.title("ROC curve comparison");

## Evaluace regresoru
Regresor můžeme vyhodnotit několika způsoby. Ukážeme si, jak vytvořit graf zobrazující reálné a predikované hodnoty, jak změnit styl pandas `DataFrame`, abychom jej mohli použít jako vizualizaci, a v neposlední řadě přidáme pár vizualizací distribuce chyby modelu.

### 🏠🏚 Graf reálných a predikovaných hodnot
K vykreslení potřebujeme vektor reálných a predikovaných hodnot. Reálná hodnota se stane x-ovou souřadnicí a predikovaná y-ovou souřadnicí. Kdyby se predikce a reálné hodnoty shodovaly, platilo by _y=x_. Proto do grafu přidáme i přímku _y=x_. Čím blíže jsou body této přímce, tím lepší model je.

In [ ]:
from numpy.random import default_rng
rng = np.random.default_rng(300)
y_real = 100 * rng.random(50)
y_pred = y_real + (rng.standard_normal(50) * 10)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

ax.vlines(y_real, y_real, y_pred, color=red, zorder=1)
ax.plot([0,100],[0,100], label='y = x')
ax.scatter(y_real, y_pred, color=grey)

ax.set_xlabel('Real value')
ax.set_ylabel('Predicted value')
ax.legend()
_ = ax.set_title('Error visualization')

### 🗒 Tabulka chyb
Na tabulku chyb potřebujeme vektor predikcí a vektor reálných hodnot. Následně si potřebujeme určit kategorie závažnosti chyb (měli bychom to udělat nějak rozumně s ohledem na distribuci chyby). V tomto příkladu to uděláme od oka 👀🙈. Řekněme, že závažnost (tj. severita) bude
* nízká (low), pokud je rozdíl reálné a predikované hodnoty v absolutní hodnotě menší než 5,
* vysoká (high), pokud je rozdíl reálné a predikované hodnoty v absolutní hodnotě větší než 10,
* střední (medium) ve všech ostatních případech.

Zkusme vytvořit dataset, který obsauhje reálné a predikované hodnoty, ale i rozdíl hodnot a severitu vypočítanou podle předchozích podmínek.

In [ ]:
def compute_severity(row):
    if abs(row.difference) < 5:
        return 'low'
    if abs(row.difference) > 10:
        return 'high'
    return 'medium'

df = pd.DataFrame()
df['real value'] = y_real
df['predicted value'] = y_pred
df['difference'] = y_pred - y_real
df['error severity'] = df.apply(compute_severity, axis=1)

In [ ]:
df.head()

Zkusme tabulku udělat trochu přehlednější tím, že řádky obarvíme podle závažnosti.

Funkce `highlight_col` vrátí `DataFrame` se stejným počtem řádků a sloupců jako `DataFrame`, který jí byl prodán jako parametr. V každé buňce obsahuje string s barvou pozadí, které by mělo být aplikováno.

Více o tom, jak můžete stylovat pandas tabulky, si můžete přečíst v 👨🏽‍💻 [user guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html).

In [ ]:
def highlight_col(df):
    # original data are not changed
    copy = df.copy()
    
    copy[copy['error severity'] == 'low'] = 'background-color: #A5FDD3'
    copy[copy['error severity'] == 'medium'] = 'background-color: #FFEBA5'
    copy[copy['error severity'] == 'high'] = 'background-color: #FFC2C2'
        
    return copy   

df.style.apply(highlight_col, axis=None)

### 📉 Grafy distribuce chyby

Pojďme si sestrojit graf distribuce chyby. Opět si nagenerujeme nějaká data včetně predikcí, ze kterých napočítáme chyby. Jejich rozdělení si následně vizualizujeme.⚙️

In [ ]:
# data generation
rng = np.random.default_rng(300)
y_train_real = 100 * rng.random(50)
y_train_pred = y_train_real + (rng.standard_normal(50) * 10)

rng = np.random.default_rng(301)
y_test_real = 100 * rng.random(50)
y_test_pred = y_test_real + (rng.standard_normal(50) * 20)

In [ ]:
# dataframe creation
train = pd.DataFrame()
train['error'] = y_train_real - y_train_pred
train['batch'] = 'train'

test = pd.DataFrame()
test['error'] = y_test_real - y_test_pred
test['batch'] = 'test'

df = pd.concat([train, test])
df

Grafy budeme vytvářet pomocí balíčku `seaborn`. Velice jednoduchou a efektní vizualizaci chyby můžeme realizovat pomocí [box plotu](https://seaborn.pydata.org/generated/seaborn.boxplot.html) nebo [violin plotu](https://seaborn.pydata.org/generated/seaborn.violinplot.html).

In [ ]:
import seaborn as sns
sns.boxplot(data=df, x='batch', y='error')

In [ ]:
sns.violinplot(data=df, x='batch', y='error')

Trochu komplikovanější vizualizaci lze dosáhnout pomocí funkce [`displot`](https://seaborn.pydata.org/generated/seaborn.displot.html). Ta defaultně zobrazí histogram, ale pokud přidáme parametr `kde=True`, spolu s histogramem se zobrazí i kernel density estimate křivka. Pokud chceme mít přehled o jednotlivých chybách, je třeba nastavit `rug=True`. V tom případě se do grafu nad x-ovou osu přidají indikátory jednotlivých chyb. Jejich x-ová souřadnice je shodná s velikostí chyby.

In [ ]:
sns.displot(data=df, x='error', hue='batch', kde = True, rug = True)

# 🎉 A to je k evaluaci vše! 🎉 